In [1]:
import requests
import json
import collections
import numpy as np
import pandas as pd
import urllib.request
import sqlite3
import datetime
from IPython.core.interactiveshell import InteractiveShell #This package and the command below allow for the Jupyter notebook
#to print all the outputs and NOT just the final row
InteractiveShell.ast_node_interactivity = "all" #this command allows all outputs to be printed, not just the last command

In [57]:
#the data for COVID cases is coming from the GitHub repository https://github.com/CSSEGISandData/COVID-19
#the first date where we have full county data is as of 03/22/2020

date_range = pd.date_range(start="03-22-2020",end="03-22-2020")
date_range_clean = date_range.strftime('%m-%d-%Y')
updated_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+str(date_range_clean[0])+'.csv'
raw_covid_data = pd.read_csv(updated_url)
covid_data_df = pd.DataFrame(raw_covid_data)
covid_data_df = covid_data_df.dropna()

#this will become my master dataset
master_covid_data = covid_data_df


date_range = pd.date_range(start="03-23-2020",end="11-22-2020")
date_range_clean = date_range.strftime('%m-%d-%Y')
for i in date_range_clean:
#    updated_url_1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+str(i)+'.csv'
    master_covid_data = master_covid_data.append(pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+str(i)+'.csv')).dropna())
#    raw_covid_data_1 = pd.read_csv(updated_url_1)
#    covid_data_df_1 = pd.DataFrame(raw_covid_data_1)
#    covid_data_df_1 = covid_data_df_1.dropna()
#    master_covid_data.append(covid_data_df_1)


master_covid_data.describe()
master_covid_data.dtypes
master_covid_data.head()

#-- BELOW YOU CAN FIND ANOTHER REPOSITORY FOR THE SAME DATA

#covid_github_Data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
#covid_github_Data = covid_github_Data.T
#print(github_Data.head())

,FIPS,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,Incident_Rate,Case_Fatality_Ratio
count,748000.000000,748000.000000,748000.000000,748000.000000,748000.000000,748000.0,748000.000000,512232.000000,512232.000000,44690.000000,44690.000000
mean,30819.197719,38.066539,-91.425782,1443.928493,45.825332,0.0,1389.068933,1352.344744,2.383610,3643.825711,1.837211
std,15828.468602,5.663453,12.796159,7469.061429,377.463128,0.0,7169.111723,1440.788716,3.251527,2133.539625,1.603111
min,1001.000000,17.982429,-174.159600,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,75.075075,0.000000
25%,18169.000000,34.442441,-97.543329,24.000000,0.000000,0.0,20.000000,329.199841,0.000000,2160.828174,0.767896
50%,29173.000000,38.164046,-89.603221,151.000000,2.000000,0.0,142.000000,880.218369,1.486200,3384.359406,1.471791
75%,46027.000000,41.684477,-83.004571,673.000000,15.000000,0.0,650.000000,1918.577445,3.198653,4727.295506,2.492212
max,90049.000000,69.314792,-65.288130,364520.000000,23689.000000,0.0,357082.000000,18082.002457,100.000000,19399.365790,18.181818


FIPS                   float64
Admin2                  object
Province_State          object
Country_Region          object
Last_Update             object
Lat                    float64
Long_                  float64
Confirmed                int64
Deaths                 float64
Recovered              float64
Active                 float64
Combined_Key            object
Incidence_Rate         float64
Case-Fatality_Ratio    float64
Incident_Rate          float64
Case_Fatality_Ratio    float64
dtype: object

In [64]:
#This link has all the county FIPS and mappings https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv
FIPS_URL = 'https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv'
FIPS_raw_data = pd.read_csv(FIPS_URL,encoding='latin-1')
#Passing the argument encoding='latin-1' for read_csv solved this error. Double check the documentation to understand why
FIPS_counties_states = pd.DataFrame(FIPS_raw_data)
FIPS_df = FIPS_counties_states

FIPS_df.describe()
FIPS_df.dtypes
FIPS_df.head()
#Perform joins later based on the need of the project covid_data_df_mapped = pd.merge(covid_data_df,state_names_mappings_df,left_on='state', right_on='Code')
#print(covid_data_df_mapped.head())

,fips,sumlev,region,division,state,county
count,3146.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000
mean,30380.268595,49.996818,2.668788,5.192491,30.273942,103.539930
std,15172.365155,0.178372,0.803043,1.963750,15.145834,107.702765
min,1001.000000,40.000000,1.000000,1.000000,1.000000,0.000000
25%,18175.500000,50.000000,2.000000,4.000000,18.000000,35.000000
50%,29176.000000,50.000000,3.000000,5.000000,29.000000,79.000000
75%,45082.500000,50.000000,3.000000,7.000000,45.000000,133.000000
max,56045.000000,50.000000,4.000000,9.000000,56.000000,840.000000


fips               int64
county_name       object
state_abbr        object
state_name        object
long_name         object
sumlev           float64
region           float64
division         float64
state            float64
county           float64
crosswalk         object
region_name       object
division_name     object
dtype: object

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,Autauga County,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,Baldwin County,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,Barbour County,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central
3,1007,Bibb County,AL,Alabama,Bibb County AL,50.0,3.0,6.0,1.0,7.0,3-6-1-7,South,East South Central
4,1009,Blount County,AL,Alabama,Blount County AL,50.0,3.0,6.0,1.0,9.0,3-6-1-9,South,East South Central


In [74]:
#This census data has information on the demographic build of every state e.g. the proportion of people in each age bracket
demographic_date_US_states = pd.read_csv(r'C:\Users\MarceloFuentes\Documents\Springboard\Research Project - Covid Patterns\Data by County CENSUS\ACSST1Y2019.S0101_data_with_overlays_2020-11-02T191057.csv')
demographic_date_US_states_df = pd.DataFrame(demographic_date_US_states)
updated_headers = demographic_date_US_states_df.iloc[0]
demographic_date_US_states_df = demographic_date_US_states_df[1:]
demographic_date_US_states_df.columns = updated_headers 

demographic_date_US_states_df.insert(1,'FIPS','') #this command inserts a column on the second column of my data frame
demographic_date_US_states_df['FIPS'] = demographic_date_US_states_df['id'].str.slice(start=10,stop=14)
demographic_date_US_states_df.dtypes
demographic_date_US_states_df.describe().T
demographic_date_US_states_df.head(10)

0
id                                                                                               object
FIPS                                                                                             object
Geographic Area Name                                                                             object
Estimate!!Total!!Total population                                                                object
Margin of Error!!Total!!Total population                                                         object
                                                                                                  ...  
Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio    object
Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex                               object
Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex                        object
Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!

,count,unique,top,freq
0,,,,
id,840,840,0500000US36117,1
FIPS,840,576,4003,5
Geographic Area Name,840,840,"Richmond County, New York",1
Estimate!!Total!!Total population,840,840,67006,1
Margin of Error!!Total!!Total population,840,2,*****,839
...,...,...,...,...
Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,840,1,(X),840
Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,840,1,(X),840
Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,840,1,(X),840


,id,FIPS,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under 5 years,Margin of Error!!Total!!Total population!!AGE!!Under 5 years,Estimate!!Total!!Total population!!AGE!!5 to 9 years,Margin of Error!!Total!!Total population!!AGE!!5 to 9 years,Estimate!!Total!!Total population!!AGE!!10 to 14 years,...,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Age dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Age dependency ratio,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Old-age dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Old-age dependency ratio,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age
1,0500000US01003,1003,"Baldwin County, Alabama",223234,*****,10616,926,12826,2513,14373,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,0500000US01015,1015,"Calhoun County, Alabama",113605,*****,6699,200,5534,1301,7774,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,0500000US01043,1043,"Cullman County, Alabama",83768,*****,5310,610,4563,972,5906,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,0500000US01049,1049,"DeKalb County, Alabama",71513,*****,4578,552,4292,994,5519,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
5,0500000US01051,1051,"Elmore County, Alabama",81209,*****,4272,557,6638,1274,3812,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
6,0500000US01055,1055,"Etowah County, Alabama",102268,*****,5649,524,5923,1170,7428,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
7,0500000US01069,1069,"Houston County, Alabama",105882,*****,6637,37,5847,549,7545,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
8,0500000US01073,1073,"Jefferson County, Alabama",658573,*****,42248,114,39554,2576,43536,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
9,0500000US01077,1077,"Lauderdale County, Alabama",92729,*****,3962,390,3711,840,6862,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
10,0500000US01081,1081,"Lee County, Alabama",164542,*****,8512,2505,12662,2744,7503,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [27]:
#This is the political map of 2016 based on per-county data. The counties' unique IDs are under the column called "combined_fips"
electoral_map_2016 = pd.read_csv('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-20/master/2016_US_County_Level_Presidential_Results.csv')
electoral_map_2016_df = pd.DataFrame(electoral_map_2016)
electoral_map_2016_df.dtypes
electoral_map_2016_df.describe()
electoral_map_2016_df.head(10)

Unnamed: 0          int64
votes_dem         float64
votes_gop         float64
total_votes       float64
per_dem           float64
per_gop           float64
diff               object
per_point_diff     object
state_abbr         object
county_name        object
combined_fips       int64
dtype: object

,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,combined_fips
count,3141.000000,3.141000e+03,3141.000000,3.141000e+03,3141.000000,3141.000000,3141.000000
mean,1570.000000,2.073410e+04,20645.278574,4.363662e+04,0.317637,0.635144,30388.584527
std,906.872924,7.200370e+04,41627.228433,1.145680e+05,0.152983,0.156130,15162.375610
min,0.000000,4.000000e+00,57.000000,6.400000e+01,0.031447,0.041221,1001.000000
25%,785.000000,1.175000e+03,3241.000000,4.870000e+03,0.205370,0.545760,18179.000000
50%,1570.000000,3.194000e+03,7268.000000,1.114400e+04,0.286400,0.665353,29177.000000
75%,2355.000000,1.004700e+04,18130.000000,2.979900e+04,0.398241,0.750265,45081.000000
max,3140.000000,1.893770e+06,620285.000000,2.652072e+06,0.928466,0.952727,56045.000000


,Unnamed: 0,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,0,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,1,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,2,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,3,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,4,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060
5,5,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2068
6,6,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2070
7,7,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2090
8,8,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2100
9,9,93003.0,130413.0,246588.0,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2105


In [87]:
#the data is coming from https://chronicdata.cdc.gov/Nutrition-Physical-Activity-and-Obesity/Nutrition-Physical-Activity-and-Obesity-Behavioral/hn4x-zwk7/data
CDC_raw = pd.read_csv(r'C:\Users\MarceloFuentes\Documents\Springboard\Research Project - Covid Patterns\CDC Data on obesity\Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv')
CDC_data_df = pd.DataFrame(CDC_raw)
CDC_data_df = CDC_data_df[CDC_data_df['YearEnd']==2018]
#using the most recent data for the study which is from 2018
CDC_data_df.dtypes
CDC_data_df.describe()
CDC_data_df.head(10)
#the data from the CDC only covers 33 US states

YearStart                       int64
YearEnd                         int64
LocationAbbr                   object
LocationDesc                   object
Datasource                     object
Class                          object
Topic                          object
Question                       object
Data_Value_Unit               float64
Data_Value_Type                object
Data_Value                    float64
Data_Value_Alt                float64
Data_Value_Footnote_Symbol     object
Data_Value_Footnote            object
Low_Confidence_Limit          float64
High_Confidence_Limit         float64
Sample_Size                    object
Total                          object
Age(years)                     object
Education                      object
Gender                         object
Income                         object
Race/Ethnicity                 object
GeoLocation                    object
ClassID                        object
TopicID                        object
QuestionID  

,YearStart,YearEnd,Data_Value_Unit,Data_Value,Data_Value_Alt,Low_Confidence_Limit,High_Confidence_Limit,LocationID
count,4620.0,4620.0,0.0,4115.000000,4115.000000,4115.000000,4115.000000,4620.000000
mean,2018.0,2018.0,NaN,30.444787,30.444787,26.147120,35.322722,31.854545
std,0.0,0.0,NaN,7.841793,7.841793,7.761974,8.907665,18.394102
min,2018.0,2018.0,NaN,2.600000,2.600000,0.800000,7.500000,1.000000
25%,2018.0,2018.0,NaN,25.600000,25.600000,21.100000,30.000000,17.000000
50%,2018.0,2018.0,NaN,31.600000,31.600000,27.200000,36.200000,31.000000
75%,2018.0,2018.0,NaN,35.900000,35.900000,32.000000,40.900000,46.000000
max,2018.0,2018.0,NaN,60.400000,60.400000,55.900000,76.000000,78.000000


,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
727,2018,2018,CA,California,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(37.63864012300047, -120.99999953799971)",PA,PA1,Q047,VALUE,6,Gender,Male,GEN,MALE
5400,2018,2018,IN,Indiana,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(39.766910452000445, -86.14996019399968)",PA,PA1,Q047,VALUE,18,Gender,Female,GEN,FEMALE
6826,2018,2018,LA,Louisiana,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(31.31266064400046, -92.44568007099969)",PA,PA1,Q047,VALUE,22,Income,"$35,000 - $49,999",INC,INC3550
7194,2018,2018,DE,Delaware,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(39.008830667000495, -75.57774116799965)",OWS,OWS1,Q037,VALUE,10,Age (years),65 or older,AGEYR,AGEYR65PLUS
7358,2018,2018,MT,Montana,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(47.06652897200047, -109.42442064499971)",OWS,OWS1,Q037,VALUE,30,Gender,Male,GEN,MALE
7458,2018,2018,PA,Pennsylvania,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(40.79373015200048, -77.86070029399963)",OWS,OWS1,Q037,VALUE,42,Gender,Male,GEN,MALE
7722,2018,2018,LA,Louisiana,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(31.31266064400046, -92.44568007099969)",OWS,OWS1,Q036,VALUE,22,Gender,Male,GEN,MALE
7853,2018,2018,OH,Ohio,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(40.06021014100048, -82.40426005599966)",OWS,OWS1,Q036,VALUE,39,Income,"Less than $15,000",INC,INCLESS15
8011,2018,2018,MA,Massachusetts,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(42.27687047000046, -72.08269067499964)",OWS,OWS1,Q037,VALUE,25,Education,High school graduate,EDU,EDUHSGRAD
8012,2018,2018,MD,Maryland,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,"(39.29058096400047, -76.60926011099963)",PA,PA1,Q047,VALUE,24,Race/Ethnicity,American Indian/Alaska Native,RACE,RACENAA
